In [1]:
import os
import math
import shutil
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
# Define the directories where the data is stored for training, validation, and testing
train_dir = '/kaggle/input/gold-coin-dataset2/train_data_directory/'
val_dir = '/kaggle/input/gold-coin-dataset2/validation_data_directory/'
test_dir = '/kaggle/input/gold-coin-dataset2/test_data_directory'

# Define the batch size and number of epochs for training
batch_size = 16
epochs = 10

# Define image dimensions
img_width, img_height = 224, 224

In [3]:
# Get the class names from the subdirectories of the training directory
class_names = sorted(os.listdir(train_dir))
class_names

['rsbl_100_995',
 'rsbl_100_999',
 'rsbl_10_995',
 'rsbl_10_999',
 'rsbl_1_995',
 'rsbl_1_999',
 'rsbl_20_995',
 'rsbl_20_999',
 'rsbl_2_995',
 'rsbl_2_999',
 'rsbl_50_995',
 'rsbl_50_999',
 'rsbl_5_995',
 'rsbl_5_999',
 'rsbl_8_995',
 'rsbl_8_999']

In [4]:
# Set up the data generator for training images with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [5]:
# Set up the data generator for validation and testing images (only rescaling)
validation_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names
)

Found 2585 images belonging to 16 classes.


In [7]:
# Load validation data from the directory with images
validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names  
)

Found 323 images belonging to 16 classes.


In [8]:
# Load testing data from the directory with images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    classes=class_names
)

Found 360 images belonging to 16 classes.


In [9]:
strategy = tf.distribute.MirroredStrategy()

In [17]:
from keras.models import *
from keras.layers import *
from keras.applications import VGG16

VGG = VGG16(input_shape = (224,224, 3), weights = 'imagenet', include_top = False)
   
VGG.trainable = False

model = Sequential()
model.add(VGG)
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(len(class_names), activation = 'sigmoid'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

Epoch 1/10
81/81 [==============================] - 281s 3s/step - loss: 0.8283 - accuracy: 0.7729 - val_loss: 0.2901 - val_accuracy: 0.9288
Epoch 2/10
81/81 [==============================] - 194s 2s/step - loss: 0.2053 - accuracy: 0.9718 - val_loss: 0.1676 - val_accuracy: 0.9659
Epoch 3/10
81/81 [==============================] - 193s 2s/step - loss: 0.1105 - accuracy: 0.9849 - val_loss: 0.0728 - val_accuracy: 0.9938
Epoch 4/10
81/81 [==============================] - 202s 2s/step - loss: 0.0614 - accuracy: 0.9919 - val_loss: 0.1260 - val_accuracy: 0.9721
Epoch 5/10
75/81 [==========================>...] - ETA: 13s - loss: 0.0459 - accuracy: 0.9942